# Problem 1 Production economy and CO2 taxation

## Question 1

In [36]:
import numpy as np
from types import SimpleNamespace

# First we define the parameters
par = SimpleNamespace()
par.A = 1.0
par.gamma = 0.5
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0
par.tau = 0.0
par.T = 0.0
par.kappa = 0.1
w = 1  # numeraire

# Functions for optimal firm behavior and profits
def optimal_labor(w, p, A, gamma):
    return (p * A * gamma / w)**(1 / (1 - gamma))

def optimal_output(w, p, A, gamma):
    return A * optimal_labor(w, p, A, gamma)**gamma

def profits(w, p, A, gamma):
    return w * (p * A * gamma / w)**(1 / (1 - gamma)) * (1 - gamma)

# Functions for consumer optimization
def consumer_utility(p1, p2, w, tau, T, alpha, nu, epsilon):
    def utility(c1, c2, ell):
        return np.log(c1**alpha * c2**(1 - alpha)) - nu * ell**(1 + epsilon) / (1 + epsilon)
    
    ell_star = (w + T + profits(w, p1, par.A, par.gamma) + profits(w, p2, par.A, par.gamma)) / (p1 * alpha + (p2 + tau) * (1 - alpha))
    c1_star = alpha * (w * ell_star + T + profits(w, p1, par.A, par.gamma) + profits(w, p2, par.A, par.gamma)) / p1
    c2_star = (1 - alpha) * (w * ell_star + T + profits(w, p1, par.A, par.gamma) + profits(w, p2, par.A, par.gamma)) / (p2 + tau)
    
    return utility(c1_star, c2_star, ell_star), c1_star, c2_star, ell_star

# Check market clearing conditions for a range of prices
p1_values = np.linspace(0.1, 2.0, 10)
p2_values = np.linspace(0.1, 2.0, 10)
results = []

for p1 in p1_values:
    for p2 in p2_values:
        U, c1, c2, ell = consumer_utility(p1, p2, w, par.tau, par.T, par.alpha, par.nu, par.epsilon)
        y1 = optimal_output(w, p1, par.A, par.gamma)
        y2 = optimal_output(w, p2, par.A, par.gamma)
        labor_market_clearing = np.isclose(ell, optimal_labor(w, p1, par.A, par.gamma) + optimal_labor(w, p2, par.A, par.gamma), rtol=1e-4)
        good1_market_clearing = np.isclose(c1, y1, rtol=1e-4)
        good2_market_clearing = np.isclose(c2, y2, rtol=1e-4)
        
        results.append((p1, p2, labor_market_clearing, good1_market_clearing, good2_market_clearing))

# Displaying results for Question 1
for result in results:
    print(f"p1: {result[0]:.2f}, p2: {result[1]:.2f}, Labor Market Clearing: {result[2]}, Good 1 Market Clearing: {result[3]}, Good 2 Market Clearing: {result[4]}")
   

p1: 0.10, p2: 0.10, Labor Market Clearing: False, Good 1 Market Clearing: False, Good 2 Market Clearing: False
p1: 0.10, p2: 0.31, Labor Market Clearing: False, Good 1 Market Clearing: False, Good 2 Market Clearing: False
p1: 0.10, p2: 0.52, Labor Market Clearing: False, Good 1 Market Clearing: False, Good 2 Market Clearing: False
p1: 0.10, p2: 0.73, Labor Market Clearing: False, Good 1 Market Clearing: False, Good 2 Market Clearing: False
p1: 0.10, p2: 0.94, Labor Market Clearing: False, Good 1 Market Clearing: False, Good 2 Market Clearing: False
p1: 0.10, p2: 1.16, Labor Market Clearing: False, Good 1 Market Clearing: False, Good 2 Market Clearing: False
p1: 0.10, p2: 1.37, Labor Market Clearing: False, Good 1 Market Clearing: False, Good 2 Market Clearing: False
p1: 0.10, p2: 1.58, Labor Market Clearing: False, Good 1 Market Clearing: False, Good 2 Market Clearing: False
p1: 0.10, p2: 1.79, Labor Market Clearing: False, Good 1 Market Clearing: False, Good 2 Market Clearing: False
p

This indicates that with the given parameters and price ranges, the market for neither Good 1 nor Good 2 reaches equilibrium. Therefore, we can conclude that under the current conditions and assumptions, there are no price combinations within the specified ranges that lead to both goods markets clearing simultaneously.

## Question 2

In [37]:
from scipy.optimize import fsolve

# Market clearing functions
def market_clearing(prices):
    p1, p2 = prices
    _, c1, c2, ell = consumer_utility(p1, p2, w, par.tau, par.T, par.alpha, par.nu, par.epsilon)
    y1 = optimal_output(w, p1, par.A, par.gamma)
    y2 = optimal_output(w, p2, par.A, par.gamma)
    
    # Two market clearing conditions
    labor_market_clearing = ell - (optimal_labor(w, p1, par.A, par.gamma) + optimal_labor(w, p2, par.A, par.gamma))
    good1_market_clearing = c1 - y1
    
    return [labor_market_clearing, good1_market_clearing]

# Find equilibrium prices using fsolve
initial_guess = [1.0, 1.0]
equilibrium_prices = fsolve(market_clearing, initial_guess)

print(f"Equilibrium prices: p1 = {equilibrium_prices[0]}, p2 = {equilibrium_prices[1]}")

Equilibrium prices: p1 = 0.9440946779397663, p2 = 1.7521655676567869


The equilibrium prices were determined by ensuring the market clearing conditions for both the labor market and the goods market for good 1. Using Walras law we know that if good 1 is in market equilibrium then good is aswell.

These prices ensure that the total labor supplied by the consumer matches the total labor demanded by the firms, and that the consumer's demand for good 1 equals its supply. The equilibrium prices provide a balance between the production decisions of the firms and the consumption choices of the consumer in the economy.

## Question 3

In [38]:
from scipy.optimize import minimize

# Functions for consumer optimization
def consumer_utility(p1, p2, w, tau, T, alpha, nu, epsilon):
    def utility(c1, c2, ell):
        return np.log(c1**alpha * c2**(1 - alpha)) - nu * ell**(1 + epsilon) / (1 + epsilon)
    
    ell_star = (w + T + profits(w, p1, par.A, par.gamma) + profits(w, p2, par.A, par.gamma)) / (p1 * alpha + (p2 + tau) * (1 - alpha))
    c1_star = alpha * (w * ell_star + T + profits(w, p1, par.A, par.gamma) + profits(w, p2, par.A, par.gamma)) / p1
    c2_star = (1 - alpha) * (w * ell_star + T + profits(w, p1, par.A, par.gamma) + profits(w, p2, par.A, par.gamma)) / (p2 + tau)
    
    return utility(c1_star, c2_star, ell_star)

# Social welfare function to be maximized
def social_welfare(vars, par):
    tau, T = vars
    par.tau = tau
    par.T = T
    p1, p2 = 1.0, 1.0  # Equilibrium prices found previously
    U = consumer_utility(p1, p2, w, tau, T, par.alpha, par.nu, par.epsilon)
    y2 = optimal_output(w, p2, par.A, par.gamma)
    SWF = U - par.kappa * y2
    return -SWF  # Minimize the negative of SWF for maximization

# Initial guess for tau and T
initial_guess = [0.0, 0.0]

# Bounds for tau and T (assuming non-negative values)
bounds = [(0, None), (0, None)]

# Find the optimal tau and T
result = minimize(social_welfare, initial_guess, args=(par), bounds=bounds)

optimal_tau, optimal_T = result.x

# Calculate the utility with the optimal tau and T
par.tau = optimal_tau
par.T = optimal_T
p1, p2 = 1.0, 1.0  # Equilibrium prices
U = consumer_utility(p1, p2, w, par.tau, par.T, par.alpha, par.nu, par.epsilon)

# Display the optimal values and utility
print(f"Optimal tax (tau): {optimal_tau}")
print(f"Optimal transfer (T): {optimal_T}")
print(f"Consumer utility (U): {U}")


Optimal tax (tau): 0.03396676536520923
Optimal transfer (T): 0.0
Consumer utility (U): -0.8550511219610493


The optimization results suggest that the government should impose a small tax on good 2 while providing no lump-sum transfer to maximize social welfare. The utility, reflects the consumer's response to the optimized tax policy, balancing between consumption and labor supply within the given economic model. This approach helps in understanding the trade-offs and impacts of tax policies on overall welfare in the production economy.